In [1]:
from pipeNetwork import pipeNetwork
from containerClass import container
import numpy as np
from scipy.integrate import solve_ivp
import igraph
from calcTw import calcTw
from calcQ import calcQ
from containerClass import container
#from systemDefinitions import system,nodes,nodes1,pipeSections,pipeSections1
from systemDefinitions import system,pipeSections,pipeSections1, pipeSections3,orificeDiam3,pipeSections4

In [2]:
net = pipeNetwork()

In [3]:
net.addSystem(system)

In [4]:
net.addAllPipes(pipeSections3, orificeDiam3)

In [5]:
net.topoSummary()

IGRAPH DN-T 15 14 -- 
+ attr: agent (g), cyl_pressure (g), cyl_size (g), cyl_valve_type (g), discharge_time (g), D (v), M (v), MFR (v), P (v), P0 (v), T (v), T0 (v), name (v), rho (v), type (v), x (v), y (v), z (v), Cpl (e), D (e), Dtrp (e), Elb (e), H (e), L (e), Ptap (e), SV (e), Sch (e), Stee (e), Ttee (e), f (e)
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (4, 6), (5, 7), (5, 8), (8, 9), (7, 10), (10, 11), (7, 12), (12, 13), (6, 14)]
(0, 11, 8)
1 0 1 1 0
2 1 2 1 1
3 2 2 1 1
4 3 2 1 1
5 4 3 2 1
6 5 3 2 1
34 6 2 1 1
7 7 3 2 1
35 8 2 1 1
305 9 1 0 1
31 10 2 1 1
301 11 1 0 1
32 12 2 1 1
302 13 1 0 1
304 14 1 0 1


In [6]:
net.t.vs[12].successors()

[igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 13, {'type': None, 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0.128, 'MFR': None, 'name': 302})]

In [7]:
net.t.vs.select(_outdegree = 0)['type'] = 'nozzle'
for i in net.t.vs.select(_outdegree = 0):
    print(i)

igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 9, {'type': 'nozzle', 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0.25, 'MFR': None, 'name': 305})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 11, {'type': 'nozzle', 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0.28125, 'MFR': None, 'name': 301})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 13, {'type': 'nozzle', 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0.128, 'MFR': None, 'name': 302})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 14, {'type': 'nozzle', 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0.344, 'MFR': None, 'name': 304})


In [8]:
net.t.vs.select(_outdegree = 2)['type'] = 'tee'
for i in net.t.vs.select(_outdegree = 2):
    print(i)

igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 4, {'type': 'tee', 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0, 'MFR': None, 'name': 5})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 5, {'type': 'tee', 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0, 'MFR': None, 'name': 6})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 7, {'type': 'tee', 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0, 'MFR': None, 'name': 7})


In [9]:
net.t.vs.select(_indegree = 0)['type'] = 'tank'
for i in net.t.vs.select(_indegree = 0):
    print(i)

igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 0, {'type': 'tank', 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0, 'MFR': None, 'name': 1})


In [10]:
for i in net.t.vs.select(_indegree = 1 , _outdegree = 1):
    print(i)

igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 1, {'type': None, 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0, 'MFR': None, 'name': 2})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 2, {'type': None, 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0, 'MFR': None, 'name': 3})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 3, {'type': None, 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0, 'MFR': None, 'name': 4})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 6, {'type': None, 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0': None, 'P': None, 'rho': None, 'D': 0, 'MFR': None, 'name': 34})
igraph.Vertex(<igraph.Graph object at 0x0000015EFC19D408>, 8, {'type': None, 'x': None, 'y': None, 'z': None, 'M': None, 'T0': None, 'T': None, 'P0

In [11]:
net.t.edge_connectivity(0,3)

1

In [12]:
nozzles = net.t.vs.select(_outdegree = 0)
nozzles.indices
tanks = net.t.vs.select(_indegree = 0)
tanks.indices

[0]

In [13]:
firstTank = net.t.farthest_points(directed=True)[0]

In [14]:
lastNozzle = net.t.farthest_points(directed=True)[1]

In [15]:
commonNodes = []
for node in net.t.vs:
    Common = True
    for noz in nozzles.indices:
        for tank in tanks.indices:
            if node.index != noz:
                if net.t.edge_connectivity(node.index,noz)== 0:
                    Common = False
                    break
            if node.index != tank:
                if net.t.edge_connectivity(tank,node.index)==0:
                    Common = False
                    break
    if Common == True:
        commonNodes.append(node.index)

In [16]:
commonNodes

[0, 1, 2, 3, 4]

In [17]:
def assignMFR(guess):
    for noz in nozzles.indices:
        net.t.vs[noz]['MFR'] = guess / len(nozzles)


In [20]:
if len(commonNodes)>1:
    commonNode = commonNodes[-1]
else:
    print("there should be a point between the tank and the rest of the system")

In [21]:
commonNode

4

In [24]:
firstTank

0

In [92]:
gamma = 1.4
y = gamma
def main_nozzle_forward(P01, T01, m, D_orifice):
    A_o= (np.pi*D_orifice**2/4) #Area of orifice
    A_p= (np.pi*(10*D_orifice)**2/4) #Area of pipe

    Cd= 0.5 #Coefficient of discharge, It will be less than 1
    y= 1.4  #gamma value changes with the type of gas and conditions
    #g=9.81 #gravitational constant
    R= 287.058 #for air
    B = np.sqrt(A_o/A_p) 
    
    P_01=P01
    T_01=T01
    _m=m
    
    P_1=stat_P(P_01) #these equatiions need Mach number
    T_1=stat_T(T_01)
    
    m_c= mass_critical(P_1, T_1)
    
    if _m>m_c:
        print ('Flow rate too high')
        r_f = m_c/m #reduction factor
        return r_f
    elif _m == m_c:
        print ('Flow is choked')
        P_2 = P_1*(2/(y+1))**(y/(y-1))
        P_02 = P_01 + P_2 - P_1 # need to calculate this? or P_02 is P_2 critical itself?
        return (P_02)
    else: #no choking
        A=A_o
        _B=B
        from sympy.solvers import solve
        from sympy import Symbol
        P_2 = Symbol('P_2')
        _P_2 =solve(Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(y*P_1))*A*(2*rho1*(P_1-P_2))**0.5-_m, P_2)
        #This might give complex results with imaginary part of very small value in -16 orders, find way to eliminate that.
        P_02 = P_01 + P_2 - P_1
        return (P_02) # we can even get M and P2 from here
    
def mass_crtical(P1,T1):
    P_1=P1
    T_1=T1
    y=gamma
    m_c= Cd*_A*P_1*(2/R*T_1)**0.5*(y/(y+1)*(2/(y+1))**(2/(y-1)))*0.5
    return m_c

def stat_P(P0,M):
    P_0=P0
    P_stat= P0/(1+(y-1)/2*M**2)**(y/(y-1))
    return P_stat

def stat_T(T0):
    T_0=T0
    T_stat= T_0/((1+(y-1)/2 *M**2)**-1)
    return T_stat

#externally calculated Expansion factor, can be used if required.

def Exp(B, P1, P2):
    _B=B
    P_1=P1
    P_2=P2
    _gamma=gamma
    Y = 1-(0.333+1.145(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(_gamma*P_1)
    return Y

# Externally measured P02

def downstream(m, m_c, P1, P01): #Calculate P02 i.e Stagnation pressure downstream.
    P_1=P1
    P_01=P01
    y=gamma
    
    if m>m_c:
        print ('Flow rate too high')
        r_f = m_c/m #reduction factor
        return r_f
    elif m==m_c:
        print ('Flow is choked')
        P_2 = P_1*(2/(y+1))**(y/(y-1))
        P_02 = P_01 + P_2 - P_1
        return (P_02)
    else:
        A=A_o
        _m=m
        from sympy.solvers import solve
        from sympy import Symbol
        P_2 = Symbol('P_2')
        _P_2 =solve(Cd*(1-(0.333+1.145*(_B**2+0.7*_B**5+ 12*_B**13))*(P_1-P_2)/(gamma*P_1))*A*(2*rho1*(P_1-P_2))**0.5-_m, P_2)
        P_02 = P_01 + P_2 - P_1
        return (P_02)

In [93]:
_dt = 1
_initialBackP = 1e5
_initialMass = 32.2
_cylInitialTemp = 295
_agentInitialTemp = 294
_orificeDiam = 0.006
cont = [0] * len(tanks)
cont[firstTank] = container(agentInitialTemp = _agentInitialTemp,
             initialMass = _initialMass,
             cylInitialTemp = _cylInitialTemp,
             cylLength = 1.8,
             cylDiam = 0.28,
             nozzleDiam = _orificeDiam,
             wallThickness = 0.005,
             ambientTemp = 300,
             backPressure = _initialBackP,
             endTime = _dt,
             numTimeSteps = 2000)
cont[firstTank].solve()

In [94]:
mdot0 = cont[firstTank].ma()[0] - cont[firstTank].ma()[-1]

In [95]:
mdot0

1.5781868293180423

In [96]:
cont[firstTank].Pt()[0]

24513574.851490695

In [97]:
#total pressure inside the tank after dt time
P01 = cont[firstTank].Pt()[-1]

In [98]:
#total temperature inside the tank after dt time
T01 = cont[firstTank].Ta()[-1]

In [99]:
P02 = main_nozzle_forward(P01,T01,mdot0,_orificeDiam)

TypeError: stat_P() missing 1 required positional argument: 'M'

In [91]:
M = 0.1
stat_P(24513574, M)

24342748.33743746